In [165]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import yfinance as yf
import mplfinance as mpf
import requests
import plotly.graph_objects as go
import talib

In [166]:
def get_tsla_recent_data():
    session = requests.Session()
    session.headers.update({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'})
    tsla = yf.Ticker('TSLA')
    df = tsla.history(period='4mo', interval='1d')
    return df

def detect_morning_star(df):
    df = df.copy()
    #df['morning_star'] = ta.cdl.morningstar(df['Open'], df['High'], df['Low'], df['Close'])
    #df['morning_star'] = ta.cdl_pattern(df['Open'], df['High'], df['Low'], df['Close'], name="morningstar")
    df['CDLENGULFING'] = talib.CDLENGULFING(df['Open'], df['High'], df['Low'], df['Close'])
    
    return df[df['morning_star'] != 0][['Open', 'High', 'Low', 'Close', 'morning_star','Engulfing']]

def detect_morning_star_talib(df):
    df['ENGULFING'] = talib.CDLMORNINGSTAR(df['Open'], df['High'], df['Low'], df['Close'])
    return df

def detect_morning_star(df, threshold=100):
    """
    Detect Morning Star patterns in price data using TA-Lib.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing OHLC price data. Must have columns: 'Open', 'High', 'Low', 'Close'
    threshold : int, optional (default=100)
        The threshold value for pattern strength. Higher values mean more strict detection.
        
    Returns:
    --------
    pandas.Series
        Series with boolean values where True indicates a Morning Star pattern.
    """
    # Convert DataFrame to numpy arrays required by TA-Lib
    open_data = df['Open'].values
    high_data = df['High'].values
    low_data = df['Low'].values
    close_data = df['Close'].values
    
    # Use talib to detect Morning Star pattern
    # integer value represents the strength of the pattern
    # positive values indicate bullish signals, negative values indicate bearish signals
    integer_result = talib.CDLMORNINGSTAR(open_data, high_data, low_data, close_data, penetration=0.3)
    
    # Convert to boolean Series where True means pattern detected
    result = pd.Series(integer_result >= threshold, index=df.index)
    
    return result

def fake_data():
    data = {
    'Open':  [100, 80, 75, 73, 85, 90],
    'High':  [101, 82, 76, 79, 95, 96],
    'Low':   [80, 75, 70, 72, 84, 89], 
    'Close': [80, 76, 74, 85, 94, 95],
    }
    df = pd.DataFrame(data)
    return df

In [167]:
df = fake_data()

In [168]:
signals = detect_morning_star(df)
signals

Exception: input array type is not double

In [ ]:
count = (signals['ENGULFING'] != 0).sum()
print(f"Number of Morning Star patterns detected: {count}")